In [2]:
#imports and setup
import numpy as np
import cv2 as cv
import os

In [3]:
'''loading video names into a list for easier access with absolute path,
video names can be access by calling the index of the video_lst and will contain 
the current absolute path, asumes that the video folder is in the same folder as this file'''

video_path = "videos"
full_path = os.path.abspath(video_path)

video_lst = []
v_dir = os.listdir(full_path)
for name in v_dir:
    video_lst.append(full_path+'/'+name)
print(video_lst[0]) 

/home/wolf/jku/ws2022/computer_vision/test_code/videos/video_023.mp4


In [4]:
import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids


In [5]:
'''Testing Frame differencing code based on:
https://github.com/infoaryan/OPENCV-PYTHON-Zero-to-One-Course-Resources/blob/master/Video%2031%20-%20Frame%20Differencing/frame_differencing.py
Expanded with: blur, dilation and morphological noise reduction and contour finding'''

#video to capture
video = video_lst[1]

kernel = np.ones((7,7)) #kerne for dilation, erosion
kernel_blur = (3,3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (9, 9)) #kernel for morphology
THRESHOLD = 20 #when to detect difference
VALUE = 255 #which value to assign to difference

#threshold to calculate
threshold_method = cv.THRESH_BINARY
tracker = EuclideanDistTracker()

# Compute the frame difference
def frame_diff(prev_frame, cur_frame, next_frame):
    diff_frames1 = cv.absdiff(next_frame, cur_frame)
    # Absolute difference between current frame and previous frame
    diff_frames2 = cv.absdiff(cur_frame, prev_frame)
    # Return the result of bitwise 'AND' between the above two resultant images
    #gives better result than simple substraction
    return cv.bitwise_and(diff_frames1, diff_frames2)

def get_frame(cap):
    ret, frame = cap.read()
    # Resize the image not needed
    '''frame = cv.resize(frame, None, fx=scaling_factor,
        fy=scaling_factor, interpolation=cv.INTER_AREA)'''
    return frame

#finds and draws contours over pixels if greater than threshold
def construct_contours(frame, contour_threshold):
    #frame = cv.dilate(frame, (1,1), iterations=1)
    contours, hierarchy = cv.findContours(frame, cv.RETR_TREE, 
                                           cv.CHAIN_APPROX_SIMPLE)
    '''cv.drawContours(frame_th, contours=contours, contourIdx=-1, 
                     color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)'''
    detections=[]
    for contour in contours:
      if cv.contourArea(contour) < contour_threshold:
        continue
      (x, y, w, h) = cv.boundingRect(contour)
      #cv.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
      detections.append([x,y,w,h])
    
    return detections


cap = cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
next_frame = get_frame(cap)

#applying blur will possibly extend with other option for preperation
def prepare_frames(frames,kernel):
    result = []
    for frame in frames:
        result.append(cv.GaussianBlur(frame, kernel, 0))
    return result[0], result[1], result[2]

# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame,cur_frame,next_frame]
    #applying preprocessing
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    #cv.imshow("Blur", cur_frame)
    
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
    #cv.imshow("Difference", frame_difference)
    
    #applying dilation
    #frame_difference = cv.erode(frame_difference, (1,1))
    frame_difference = cv.dilate(frame_difference, kernel)
    #cv.imshow("Dilation", frame_difference)
    
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #cv.imshow("Morph", frame_difference)
    
    #converting into greyscale for contour finding
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)
    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, THRESHOLD, VALUE, threshold_method)
    #cv.imshow("Threshold", frame_difference)
    
    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, 30)
    #cv.imshow("Contours", frame_th)
    
    #cv.imshow("Difference", frame_difference)
    #cv.imshow("After Threshold", frame_th)

    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x,y,w,h,id = box_id
        cv.putText(cur_frame, str(id),(x,y-15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv.rectangle(cur_frame, (x,y),(x+w, y+h), (125,255,0), 2)
        
        cv.imshow('Detection', cur_frame)
    
    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    next_frame = get_frame(cap)
    
    if cv.waitKey(25) & 0xFF == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()

QObject::moveToThread: Current thread (0x559876778d40) is not the object's thread (0x559877b94fa0).
Cannot move to target thread (0x559876778d40)

QObject::moveToThread: Current thread (0x559876778d40) is not the object's thread (0x559877b94fa0).
Cannot move to target thread (0x559876778d40)

QObject::moveToThread: Current thread (0x559876778d40) is not the object's thread (0x559877b94fa0).
Cannot move to target thread (0x559876778d40)

QObject::moveToThread: Current thread (0x559876778d40) is not the object's thread (0x559877b94fa0).
Cannot move to target thread (0x559876778d40)

QObject::moveToThread: Current thread (0x559876778d40) is not the object's thread (0x559877b94fa0).
Cannot move to target thread (0x559876778d40)

QObject::moveToThread: Current thread (0x559876778d40) is not the object's thread (0x559877b94fa0).
Cannot move to target thread (0x559876778d40)

QObject::moveToThread: Current thread (0x559876778d40) is not the object's thread (0x559877b94fa0).
Cannot move to tar

{0: (25, 277), 1: (268, 133)}
{0: (25, 277), 1: (269, 132)}
{0: (25, 275), 1: (269, 132)}
{0: (24, 274)}
{3: (179, 282)}
{3: (182, 279)}
{3: (180, 281)}
{4: (150, 303), 5: (181, 278)}
{4: (150, 303), 5: (180, 281)}
{4: (150, 303), 5: (181, 278)}
{5: (180, 280)}
{6: (176, 282)}
{7: (180, 278), 8: (151, 299)}
{9: (181, 276)}
{9: (181, 278)}
{9: (182, 278)}
{11: (168, 280)}
{11: (172, 281)}
{12: (171, 280)}
{12: (172, 279)}
{12: (178, 282)}
{12: (169, 274)}
{13: (179, 279)}
{13: (167, 276)}
{13: (174, 277)}
{13: (179, 279)}
{13: (167, 271)}
{14: (174, 275)}
{14: (168, 278)}
{14: (175, 274)}
{15: (174, 269)}
{15: (175, 273)}
{15: (175, 269)}
{16: (174, 268)}
{16: (171, 266)}
{16: (175, 266)}
{17: (176, 265)}
{17: (171, 262)}
{17: (170, 264)}
{18: (177, 263)}
{20: (182, 217)}
{21: (181, 214)}
{21: (186, 212)}
{21: (180, 212)}
{22: (183, 211)}
{23: (174, 215)}
{24: (179, 210)}
{25: (149, 200)}
{25: (145, 203)}
{26: (129, 190)}
{26: (131, 191)}
{26: (128, 188)}
{27: (128, 186)}
{27: (128, 186